# Code example - running a profit simulation

#### Maria Silva, September 2024

## Import packages

In [1]:
import os

In [2]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [3]:
from rollup import RollupSpec
from gas import GasPriceModel
from asset import AssetPriceModel
from extraction import run_arb_profit_simulation

## Define simulation paramaters

In [4]:
# Number of iterations (i.e. Monte-carlo samples)
n_iter = 10

In [5]:
# Define rollup settings
arb_pool_fee = 0.005 # The arb pool fee must be the same in both rollups!
price_A = 101.0
price_B = 100.0 # price_A must be higher than price_B!
arb_pool_reserve_x_A = 1000.0
arb_pool_reserve_x_B = 1000.0

In [6]:
# Init spec classes - rollup A
gas_price_model_A = GasPriceModel(
    model_type="gaussian", gas_price_mean=0.01, gas_price_std=0.0001
)
rollup_A = RollupSpec(
    fail_rate=0.5,
    gas_price_model=gas_price_model_A,
    gas_units_swap=10.0,
    gas_units_fail=1.0,
    arb_pool_reserve_x=arb_pool_reserve_x_A,
    arb_pool_reserve_y=arb_pool_reserve_x_A*price_A, 
    arb_pool_fee=arb_pool_fee,
)

# Init spec classes  - rollup B
gas_price_model_B = GasPriceModel(
    model_type="gaussian", gas_price_mean=0.01, gas_price_std=0.0001
)
rollup_B = RollupSpec(
    fail_rate=0.5,
    gas_price_model=gas_price_model_A,
    gas_units_swap=10.0,
    gas_units_fail=1.0,
    arb_pool_reserve_x=arb_pool_reserve_x_B,
    arb_pool_reserve_y=arb_pool_reserve_x_B*price_B,
    arb_pool_fee=arb_pool_fee,
)

In [7]:
# Define asset price settings
y_price_model = AssetPriceModel(
    asset_label="Y", fee=0.005, model_type="constant", asset_price_mean=50.0
)

## Run simulation

In [8]:
# Run simulation
arb_sim_df = run_arb_profit_simulation(n_iter, rollup_A, rollup_B, y_price_model)
arb_sim_df

,iter,fail_outcome_A,fail_outcome_B,gas_price_A,gas_price_B,pure_bundle_profit_A,pure_bundle_profit_B,atomic_bundle_profit,non_atomic_bundle_profit,atomic_arb_cost,non_atomic_arb_cost,atomic_profit,non_atomic_profit,shared_sequencing_gain
0,0,0,1,0.009983,0.010017,253.290065,0.000000,0.000000,253.290065,0.019999,0.109844,-0.019999,253.180220,-253.200219
1,1,1,0,0.010171,0.009902,0.000000,0.469704,0.000000,0.469704,0.020073,0.109194,-0.020073,0.360510,-0.380583
2,2,1,0,0.010033,0.009979,0.000000,0.469704,0.000000,0.469704,0.020012,0.109821,-0.020012,0.359883,-0.379895
3,3,0,1,0.010008,0.009900,253.290065,0.000000,0.000000,253.290065,0.019908,0.109980,-0.019908,253.180084,-253.199993
4,4,1,0,0.009825,0.010195,0.000000,0.469704,0.000000,0.469704,0.020020,0.111776,-0.020020,0.357928,-0.377948
5,5,0,0,0.010056,0.009939,253.290065,0.469704,253.759769,253.759769,0.199944,0.199944,253.559825,253.559825,0.000000
6,6,1,0,0.009921,0.010091,0.000000,0.469704,0.000000,0.469704,0.020012,0.110835,-0.020012,0.358869,-0.378881
7,7,0,1,0.010083,0.010020,253.290065,0.000000,0.000000,253.290065,0.020103,0.110852,-0.020103,253.179212,-253.199315
8,8,0,0,0.009873,0.010048,253.290065,0.469704,253.759769,253.759769,0.199206,0.199206,253.560563,253.560563,0.000000
9,9,0,1,0.009900,0.009803,253.290065,0.000000,0.000000,253.290065,0.019704,0.108806,-0.019704,253.181258,-253.200962


In [9]:
arb_sim_df["shared_sequencing_gain"].agg(["mean", "std"])

mean   -101.431780
std     130.621473
Name: shared_sequencing_gain, dtype: float64